In [1]:
import sys
import time
import subprocess
import signal
import os
import math
import rospy
import rosbag
import roslib
roslib.load_manifest("robot_comm")
from robot_comm.srv import *
roslib.load_manifest("Mocap")
from Mocap.msg import *
from Mocap.srv import *

In [169]:
get_cart = rospy.ServiceProxy('/robot_GetCartesian', robot_GetCartesian)
set_work_object = rospy.ServiceProxy('/robot_SetWorkObject', robot_SetWorkObject)
set_tool = rospy.ServiceProxy('/robot_SetTool', robot_SetTool)
set_cart = rospy.ServiceProxy('/robot_SetCartesian', robot_SetCartesian)
#Set Speed
set_speed = rospy.ServiceProxy('/robot_SetSpeed', robot_SetSpeed)
set_speed(50, 5)
get_cart()

x: 650.01
y: 0.0
z: 25.01
q0: 1.0
qx: 0.0
qy: 0.0
qz: 0.0
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [2]:
set_mocap_tf = rospy.ServiceProxy('/mocap_SetMocapTransformation', mocap_SetMocapTransformation)
set_mocap_tf(686.48, -281.16, 62.547, 0.5053, 0.50804, 0.50412, 0.48212)

ret: 1
msg: OK.

In [176]:
set_work_object(0,0,0,1,0,0,0)
set_tool(-100,0,110, 0, 0, 1, 0)
get_cart()

x: 650.04
y: 0.0
z: 25.02
q0: 1.0
qx: 0.0
qy: -0.0001
qz: 0.0
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [178]:
set_cart(650, 0, 25, 1, 0, 0, 0)

ret: 1
msg: ROBOT_CONTROLLER: OK.

In [141]:
#p = subprocess.Popen("rosbag record -O test.bag /Mocap /netft_data");

In [179]:
f1 = open("pos.txt", "w");
f2 = open("force.txt", "w");
bag = rosbag.Bag('test.bag')
last_topic = '/netft_data'
ct = 0
last_pos_x = 0
last_pos_y = 0
for topic, msg, t in bag.read_messages(topics=['/Mocap', '/netft_data']):
    if topic == '/Mocap' and last_topic == '/netft_data':
        pos = msg.body_poses[0].position
        ori = msg.body_poses[0].orientation    
        s = str(pos.x) + ' ' + str(pos.y) + ' ' + str(pos.z) + ' '
        s = s + str(ori.w) + ' ' + str(ori.x) + ' ' + str(ori.y) + ' ' + str(ori.z)
        s = s + '\n'
        f1.write(s);  
    if topic == '/netft_data' and last_topic == '/Mocap':
        force = msg.wrench.force
        s = str(force.x) + ' ' + str(force.y) + ' ' + str(force.z)
        s = s + '\n'
        f2.write(s)
    last_topic = topic
bag.close()
f1.close()
f2.close()

In [3]:
# Just dump the raw mocap body data.
f1 = open("mocap_pos_2.txt", "w")
bag = rosbag.Bag('test_human_push_2.bag')
for topic, msg, t in bag.read_messages(topics=['/Mocap']):
    if topic == '/Mocap':
        pos = msg.body_poses[0].position
        ori = msg.body_poses[0].orientation
        s = str(t.to_sec()) + '\n'
        s = s + str(pos.x) + ' ' + str(pos.y) + ' ' + str(pos.z) + ' '
        s = s + str(ori.w) + ' ' + str(ori.x) + ' ' + str(ori.y) + ' ' + str(ori.z)
        s = s + '\n'
        f1.write(s);
f1.close()

In [199]:
t.to_sec()

1429908519.3448133

In [122]:
set_tool(-100, 0, 110, 0, 0, 1, 0);


In [121]:
set_cart(550, -100, 375, 1, 0, 0, 0)

x: 549.99
y: -99.97
z: 375.0
q0: 0.0
qx: -0.7071
qy: 0.7071
qz: 0.0001
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [123]:
get_joints = rospy.ServiceProxy('/robot_GetJoints', robot_GetJoints)

In [124]:
get_joints()

j1: 0.0
j2: 20.55
j3: 0.92
j4: 0.01
j5: 68.54
j6: 90.0
ret: 1
msg: ROBOT_CONTROLLER: OK.